# Bees System Algorithm

Edoardo Bucheli A01016080
Ernesto Campos A00759359

In this notebook we present an implementation of the Bees System Algorithm to find the solution to a 2-dimensional Sensor Network problem. First we implement several helper functions and finally we present an example problem statement and solution.

Let us first talk about the characterization of the problem. The terrain is represented as a 2D grid with dimensions $6\times 6$. We represent positions with a tuple $(a,b)$ where $a$ is the row and $b$ is the column. The upper left coordinate is $(1,1)$ and the bottom right corner is the coordinate $(6,6)$.

The terrain or `board` is represented as a list that is itself composed of lists each containing the information for a row. In its original configuration, a $1$ represents a cell that we wish to have a sensor in and $0$ represents a cell which does not interests us. This changes when a solution is generated, this is explained later in this document.

We use a list `sensors` whose length is equal to the number of sensors to place in the terrain. Each nth element `sensors[n]` represents the dimension of the receptive field of the sensor. So a sensor represented by the number $3$ has a receptive field of $3\times 3$. The coordinate given is the upper left corner of said recpetive field.

A solution is represented as a list of tuples specifying the position for each sensor. This must coincide with the list `sensors`, so the first element in a list `sol` refers to the position of the first element in the `sensors` list, and so on.

In [1]:
import numpy as np
import copy

The first two functions simply take a board and print it so it can be easily interpreted by the user. We use two functions since representation for a terrain and a solution is different. To understand the difference check the text before the function `make_sol()`

In [2]:
def print_sol(board):
    """Input a solution board as made by make_sol() and print it"""
    n = len(board)
    print('  ',end = '')
    for i in range(0,n):
        print(i+1,'',end ='')
    print('')
    for i in range(0,n):
        print(i+1,end = ' ')
        for j in range(0,n):
            print(board[i][j],'',end = '')
        print('')

In [3]:
def print_board(board):
    """Print the initial board"""
    n = len(board)
    print('  ',end = '')
    for i in range(0,n):
        print(i+1,'',end ='')
    print('')
    for i in range(0,n):
        print(i+1,end = ' ')
        for j in range(0,n):
            if board[i][j] == 1:
                print('- ',end = '')
            else:
                print('X ',end = '')
        print('')

### Characterization of a solution

There are several things we wish to represent in a solution, thus the representation for the configuration of a terrain and one where the sensors have been placed are different. 

A sensor may have a receptive field greater than 1 as specified by the `sensors` list. From the position coordinate, a sensor extends as many units to the right and below as its receptive field making a square. 

A number then, represents the sensor that is acting upon the cell. If a cell contains the number $1$ that means that that cell is being recorded by sensor 1. A 2 means the same for the second sensor and so on.

If a cell is within the receptive field of a sensor but it is a cell that we do not wish to record then a '#' symbol appears instead. If a cell is not within the receptive field of any sensor but we wish to record it, its value should be '-'. Finally, 'X' means that there is no sensor and we do not wish to record the cell anyway.

The formula `make_sol()` takes an original terrain configuration, a list that represents a solution `sol` and the list `sensors` and returns a board as described in this section.

In [4]:
def make_sol(board,sol,sensors):
    """
    Take terrain configuration, a solution and a list with the sensors and their size.
    Return board with sensors placed as specified by the solution and sensor size     
    """
    for i in range(0,6):
        for j in range(0,6):
            if board[i][j]==1:
                board[i][j]='-'
            elif board[i][j]==0:
                 board[i][j]='X'
    count = 1
    for n in sol:
        i = n[0]-1
        j = n[1]-1
            
        this_sensor = sensors[count-1]
        for k in range(0,this_sensor):
            for l in range(0,this_sensor):
                if not(i+k>5 or j+l>5):
                    if(board[i+k][j+l]=='-'):
                        board[i+k][j+l]=count
                    elif(board[i+k][j+l]=='X'):
                        board[i+k][j+l]= '#'
                else:
                    pass
        count += 1
    return board

### Fitness of a solution

We measure how good a solution is by checking how many of the desired cells are within the receptive field of a sensor. Therefore we wish to maximize our fitness.

In [5]:
def checksolution(board):
    """Check the fitness of a solution, how many of the desired cells have a sensor"""
    total = 0
    for i in range(0,6):
        for j in range(0,6):
            if type(board[i][j])==int:
                if board[i][j]>0:
                    total += 1
    return total

### Bees Algorithm

The previous blocks were used for representation of the problem, this following blocks are part of the bees algorithm. A bee is represented exactly the same as a solution (a list of tuples specifying where the sensors are placed). A swarm is a list of bees. 

1. `generate_random_bees()` creates a list with randomly initialized bees within the grid.
1. `check_swarm_fitness()` takes a swarm and returns a list with the indices of the best $n$ bees where the first element of the list represents the bee with the best fitness.
1. `make_swarm()` takes the list generated by `check_swarm_fitness()` and returns a new smaller swarm organized like the fitness list.
1. `generate_scouts_2()` takes a swarm of elite or best bees, generates scouts that search around the elite and best positions and picks the best one for each location. A new swarm of the same size is returned but with the best within the scouts.

In [6]:
def generate_random_bees(n,sensors,num_bees):
    """
    Generate a new set of random bees
    Inputs: n: size of terrain (nXn)
            sensors: list with the sensors to place
            num_bees: number of random bees to generate
      Output: list with sensor positions for each bee
    """
    bees = []
    this_bee = []
    for i in range(0,num_bees):
        for k in range(0,len(sensors)):
            this_bee.append((np.random.randint(n)+1,np.random.randint(n)+1))
        bees.append(this_bee)
        this_bee = []
    return bees

In [7]:
def check_swarm_fitness(swarm,sel,board,sensors):
    """ Find the n best bees in a swarm. 
        Inputs: swarm: a swarm of bees, list with the sensor position for each bee.
                sel: how many bees to select from the swarm.
                board: the configuration of the terrain.
                sensors: list with the sensors.
    
        Return a list with the index of the best bees, first in the list is the best, 
        second in the list is second best, etc.
    """
    fitness = []
    best_bees = []
    for bee in swarm:
        this_board = copy.deepcopy(board)
        make_sol(this_board,bee,sensors)
        this_fitness = checksolution(this_board)
        fitness.append(this_fitness)
    for i in range(0,sel):
        this_best = np.argmax(fitness)
        best_bees.append(this_best)
        fitness[this_best] = 0
    return best_bees

In [8]:
def make_swarm(swarm,best):
    """Given a swarm and a list (as generated by check_swarm_fitness()) return a new, smaller swarm"""
    best_swarm=[]
    for i in range(0,len(best)):
        best_swarm.append(swarm[best[i]])
    return best_swarm  

In [9]:
def generate_scouts_2(swarm,num_scouts,board,sensors):
    """ Given a swarm of elite or best bees, generate scouts around a location and pick the best for each place
        Inputs: swarm: the list of elite/best bees
                num_scouts: how many scouts per elite/best bee
                board: the configuration of the terrain
                sensors: the list of sensors to place
        Return: A new swarm, optimized with scouts.
    """
    
    final_swarm = []
    for bee in swarm:
        mini_swarm = []
        mini_swarm.append(bee)
        for i in range(0,num_scouts):
            new_scout = []
            for sen in bee:
                pos1 = sen[0]
                if not(pos1<=1 or pos1 >=6):
                    pos1 = pos1+np.random.choice([0,-1,1],p=[0.4,0.3,0.3])
                pos2 = sen[1]
                if not(pos2<-1 or pos2>=6):  
                    pos2 = pos2+np.random.choice([0,-1,1],p=[0.4,0.3,0.3])
                new_scout.append((pos1,pos2))
        mini_swarm.append(new_scout)
        best = check_swarm_fitness(mini_swarm,1,board,sensors)[0]
        best_bee = mini_swarm[best]
        #print(best_bee)
        final_swarm.append(best_bee)
    return final_swarm

### Putting everything together

`bees_SensorNetwork()` puts everything together by creating random bees, finding elite and best bees and making new generations by taking the best and new randomly initialized bees.

In [10]:
def bees_SensorNetwork(board,sensors,num_bees,num_elite,m, generations=1000):
    """Bees System Algorithm: carry out several generations of the algorithm"""
    iteration = 1
    swarm = generate_random_bees(len(board),sensors,num_bees)
  
    while iteration < generations:
  
        best = check_swarm_fitness(swarm,m,board,sensors)
        best_swarm = make_swarm(swarm,best)
        elite = best_swarm[0:num_elite]
        next_best = best_swarm[num_elite:]
        new_gen = []

        new_gen.extend(generate_scouts_2(elite,4,board,sensors))
        new_gen.extend(generate_scouts_2(next_best,2,board,sensors))
        new_gen.extend(generate_random_bees(len(board),sensors,num_bees-m))

        swarm = new_gen
        iteration += 1
   
    best_ind = check_swarm_fitness(swarm,1,board,sensors)[0]

    return swarm[best_ind]

## Experimentation

We provide an example of the algorithm working, let's start by creating a terain, a possible solution and a set of sensors

In [11]:
board = [[1,1,0,0,1,0],[1,0,0,1,1,1],[0,0,1,1,1,1],[1,1,1,1,1,1],[0,0,0,1,1,1],[1,1,1,1,0,0]]

sol = [(1,1),(1,5),(4,5),(5,3)]

sensors = [3,2,2,1]

Let's see how the configuration looks and how a solution looks like

In [12]:
print('The board without sensors looks like this: ')
print_board(board)
print('\nThe initial solution is: ')
initial_board = copy.deepcopy(board)
make_sol(initial_board,sol,sensors)
print_sol(initial_board)
print('It has a fitness of: ',checksolution(initial_board))

The board without sensors looks like this: 
  1 2 3 4 5 6 
1 - - X X - X 
2 - X X - - - 
3 X X - - - - 
4 - - - - - - 
5 X X X - - - 
6 - - - - X X 

The initial solution is: 
  1 2 3 4 5 6 
1 1 1 # X 2 # 
2 1 # # - 2 2 
3 # # 1 - - - 
4 - - - - 3 3 
5 X X # - 3 3 
6 - - - - X X 
It has a fitness of:  11


Now we use the Bees System Algorithm to find an optimal solution. We create 10 bees, and for each generation we pick $m=4$ bees out of which 2 are elite.

In [13]:
solution = bees_SensorNetwork(board,sensors,10,2,4)

In [14]:
check_board = copy.deepcopy(board)

make_sol(check_board,solution,sensors)

print_sol(check_board)

print("Fitness: ",checksolution(check_board))

  1 2 3 4 5 6 
1 - 4 X X 3 # 
2 - X X - 3 3 
3 X # 2 1 1 1 
4 - 2 2 1 1 1 
5 X X X 1 1 1 
6 - - - - X X 
Fitness:  16


Having run this algorithm several times, we see that although the solution may be different the best possible solution of 16 is always found.

### Second Experiment
Let's try to use the same algorithm with a different board and more sensors. The board we have created for this experiment has a pretty specific optimal solution and thus several possible local optima, let's see how the algorithm handles it.

In [15]:
board2 = [[1,1,1,1,1,0],[1,1,1,1,1,0],[0,0,1,1,1,0],[0,0,1,1,1,1],[1,1,0,1,1,1],[1,1,0,1,1,1]]

sol2 = [(1,1),(1,4),(4,1),(4,3),(6,1)]

sensors2 = [3,3,2,2,1]

In [16]:
print('The board without sensors looks like this: ')
print_board(board2)

The board without sensors looks like this: 
  1 2 3 4 5 6 
1 - - - - - X 
2 - - - - - X 
3 X X - - - X 
4 X X - - - - 
5 - - X - - - 
6 - - X - - - 


In [17]:
solution2 = bees_SensorNetwork(board2,sensors2,num_bees = 50,num_elite = 5,m=10,generations=4000)

In [18]:
check_board2 = copy.deepcopy(board2)

make_sol(check_board2,solution2,sensors2)

print_sol(check_board2)

print("Fitness: ",checksolution(check_board2))

  1 2 3 4 5 6 
1 4 4 2 2 2 X 
2 4 4 2 2 2 X 
3 X X 2 2 2 X 
4 X X 5 1 1 1 
5 3 3 X 1 1 1 
6 3 3 X 1 1 1 
Fitness:  27


To get the optimal solution we had to bump the settings a little but we can see that the algorithm managed to find the optimal solution where every space has a sensor in it.